In [1]:
# tests/test_order_with_city_viewer.py
import sys, os, random, math
sys.path.insert(0, r"D:\LLMDelivery-LJ\SimWorld")
sys.path.insert(0, r"D:\LLMDelivery-LJ\LLM-Delivery")

from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QTimer

from simworld.utils.vector import Vector
from simworld.map.map import Map
from simworld.utils.city_viewer import CityViewer, build_city_map, thin_by_min_spacing_xy

from Base.Order import Order
from Base.Food import FoodItem, FoodCategory, OdorLevel
from Base.Insulated_bag import InsulatedBag

# ---------------- 基础配置 ----------------
CONFIG = {
    "map.input_roads": r"D:\LLMDelivery-LJ\Test_Data\test\roads.json",
    "traffic.sidewalk_offset": 100.0,
}
WORLD_JSON    = r"D:\LLMDelivery-LJ\Test_Data\test\progen_world_enriched.json"
BUILDING_DEFS = r"D:\LLMDelivery-LJ\Test_Data\test\buildings.json"
FOOD_JSON     = r"D:\LLMDelivery-LJ\LLM-Delivery\input\food.json"

AMBIENT_C     = 25.0            # 环境温度
RIDER_SPEED   = 2.2             # m/s（恒定；通过 time_scale 控制全局时间流速）
DT_BASE       = 0.2             # 基础 dt（秒）
INFO_EVERY    = 1.0             # 信息刷新间隔（秒）
PREP_TIME_RNG = (30.0, 120.0)   # 运行时随机准备时间（秒），覆盖 JSON 中的 prep_time_s
TIME_SCALES   = [0.5, 1.0, 2.0, 4.0]


class SimRunner:
    def __init__(self, viewer: CityViewer, city_map: Map):
        self.viewer = viewer
        self.map = city_map
        self.timer = QTimer()
        self.timer.timeout.connect(self._on_tick)

        # 放大信息区
        try:
            self.viewer.set_info_area_height(220)
            self.viewer.set_info_font_point_size(11)
        except Exception:
            pass

        # 读取 catalog（新 schema）
        self.catalog = FoodItem.load_simple_catalog(FOOD_JSON)  # {name: spec}

        # 状态
        self.order = None
        self.items = []           # List[FoodItem]（取餐前也会演化）
        self.bag = None
        self.phase = "idle"       # "to_pickup" → "to_drop" → "done"
        self.route_to_pick = []
        self.route_to_drop = []
        self.cur_route = []
        self.cur_idx = 0
        self.pos = None
        self.last_info_t = 0.0
        self.sim_time = 0.0
        self.time_scale = 1.0     # 仅改变时间流速；速度恒定
        self.speed_m_s = RIDER_SPEED

    # ---------- 生成一单并绘制 ----------
    def new_random_order(self):
        bxy = self.viewer.get_buildings_xy()
        if len(bxy) < 3:
            self.viewer.set_info_text("⚠️ 建筑数量不足（<3）。")
            return

        start  = random.choice(bxy)
        pickup = random.choice([p for p in bxy if p != start])
        drop   = random.choice([p for p in bxy if p != pickup])

        # 从 JSON 挑菜（1~3 个）
        all_names = list(self.catalog.keys())
        items_names = random.sample(all_names, k=min(len(all_names), random.randint(1, 3)))
        note = random.choice(["", "Leave at door", "Call on arrival", "No contact delivery", "Extra napkins"])

        self.order = Order(
            pickup_address=Vector(pickup[0], pickup[1]),
            delivery_address=Vector(drop[0],  drop[1]),
            items=items_names,
            special_note=note
        )
        self.order.compute_with_map(self.map)

        start_node = self.map.get_closest_node(Vector(start[0], start[1]))
        pick_node  = self.map.get_closest_node(self.order.pickup_address)
        drop_node  = self.map.get_closest_node(self.order.delivery_address)

        path1_nodes = self.map.get_shortest_path(start_node, pick_node)
        path2_nodes = self.map.get_shortest_path(pick_node,  drop_node)
        if not path1_nodes or not path2_nodes:
            self.viewer.set_info_text("❌ 路径计算失败（检查道路连通性）。")
            return

        self.route_to_pick = [(n.position.x, n.position.y) for n in path1_nodes]
        self.route_to_drop = [(n.position.x, n.position.y) for n in path2_nodes]

        r1_plot = thin_by_min_spacing_xy(self.route_to_pick, 1200.0)
        r2_plot = thin_by_min_spacing_xy(self.route_to_drop, 1200.0)
        self.viewer.clear_overlays()
        self.viewer.plot_route(r1_plot, color="#9E9E9E", width=2.0, scatter=False, show_endpoints=True)
        self.viewer.plot_route(r2_plot, color="#FF5722", width=3.0, scatter=False, show_endpoints=True)

        # 构建 FoodItem；随机设置准备时间（覆盖 JSON）
        self.items = []
        for name in items_names:
            it = FoodItem.make_from_catalog(name, self.catalog)
            it.begin_preparation(random.uniform(*PREP_TIME_RNG))
            self.items.append(it)

        # 初始化阶段
        self.phase = "to_pickup"
        self.cur_route = list(self.route_to_pick)
        self.cur_idx = 1
        self.pos = self.cur_route[0]
        self.sim_time = 0.0
        self.last_info_t = 0.0
        self.bag = None

        self._draw_rider()
        self._update_info(force=True)

    # ---------- 时间流速 ----------
    def set_time_scale(self, s: float):
        self.time_scale = max(0.1, float(s))

    # ---------- 控制 ----------
    def start(self):
        if self.order is None:
            self.new_random_order()
        self.timer.start(int(DT_BASE * 1000))

    def stop(self):
        self.timer.stop()

    # ---------- Tick ----------
    def _on_tick(self):
        if self.phase == "idle" or self.order is None:
            return

        dt = DT_BASE * self.time_scale
        self.sim_time += dt
        dist_to_go = self.speed_m_s * 100.0 * dt  # cm

        # 取餐前：已准备完成的菜在餐厅环境演化
        if self.phase == "to_pickup" and self.items:
            for it in self.items:
                it.step(dt, env_temp_C=AMBIENT_C, speed_m_s=0.0)

        # 沿折线推进
        while dist_to_go > 0 and self.cur_idx < len(self.cur_route):
            cx, cy = self.pos
            nx, ny = self.cur_route[self.cur_idx]
            dx, dy = (nx - cx), (ny - cy)
            seg_len = math.hypot(dx, dy)

            if seg_len < 1e-6:
                self.pos = (nx, ny); self.cur_idx += 1; continue

            if dist_to_go < seg_len:
                r = dist_to_go / seg_len
                self.pos = (cx + dx * r, cy + dy * r)
                dist_to_go = 0
            else:
                self.pos = (nx, ny)
                self.cur_idx += 1
                dist_to_go -= seg_len

        # 阶段切换
        if self.cur_idx >= len(self.cur_route):
            if self.phase == "to_pickup":
                self._load_bag()
                self.order.start_time = self._now()  # 超时从取餐开始算
                self.phase = "to_drop"
                self.cur_route = list(self.route_to_drop)
                self.cur_idx = 1
                self.pos = self.cur_route[0]
            elif self.phase == "to_drop":
                self.phase = "done"
                self.order.has_delivered = True
                self._update_info(force=True)
                self._draw_rider()
                self.stop()
                return

        # 送餐阶段：袋内演化
        if self.phase == "to_drop" and self.bag is not None:
            self.bag.step(dt, ambient_temp_C=AMBIENT_C, speed_m_s=self.speed_m_s)

        # 重画 + 信息节流
        self._draw_rider()
        self.last_info_t += dt
        if self.last_info_t >= INFO_EVERY:
            self._update_info()
            self.last_info_t = 0.0

    # ---------- 可视化 ----------
    def _draw_rider(self):
        r1_plot = thin_by_min_spacing_xy(self.route_to_pick, 1200.0)
        r2_plot = thin_by_min_spacing_xy(self.route_to_drop, 1200.0)
        self.viewer.clear_overlays()
        self.viewer.plot_route(r1_plot, color="#9E9E9E", width=2.0, scatter=False, show_endpoints=True)
        self.viewer.plot_route(r2_plot, color="#FF5722", width=3.0, scatter=False, show_endpoints=True)
        self.viewer.plot_route([self.pos], color="#E53935", width=0.0, scatter=True, show_endpoints=False)

    def _update_info(self, force=False):
        dist_m = self.order.distance_cm / 100.0
        eta_s  = self.order.eta_s
        tlim_s = self.order.time_limit_s
        eta_m, eta_s2   = int(eta_s // 60), int(eta_s % 60)
        tlim_m, tlim_s2 = int(tlim_s // 60), int(tlim_s % 60)

        lines = [
            f"Order ID: {self.order.id}",
            f"Pickup:  ({int(self.order.pickup_address.x)}, {int(self.order.pickup_address.y)})",
            f"Dropoff: ({int(self.order.delivery_address.x)}, {int(self.order.delivery_address.y)})",
            f"Items: {', '.join(self.order.items)}",
            f"Distance: {dist_m:.1f} m   |   Speed: {self.speed_m_s:.2f} m/s   |   Time x{self.time_scale:.1f}",
            f"ETA: ~{eta_m}m {eta_s2}s   |   Time Limit: {tlim_m}m {tlim_s2}s",
            f"Phase: {self.phase}",
        ]

        def fmt_simple(it):
            m = it.simple_metrics()
            # state: 冰淇淋→melt，其它→非热时间态
            stype = "melt" if it.category == FoodCategory.FROZEN else "state"
            return f"T={m['temp_C']:.1f}°C, damage={m['damage']:.2f}, odor={m['odor_mix']:.2f}, {stype}={m['state']:.2f}"

        if self.bag is None:
            lines.append("— Food (prep / waiting in restaurant) —")
            for it in self.items:
                state = "preparing" if it.is_preparing and not it.prepared else "waiting"
                lines.append(f"{it.name} [{it.category.name}]  {state} | {fmt_simple(it)}")
        else:
            lines.append(f"— Food (in bag, durability={self.bag.durability*100:.0f}%) —")
            for idx in range(self.bag.num_compartments):
                comp = self.bag.list_items(idx)
                if not comp:
                    continue
                has_strong = any(it.odor_level is OdorLevel.STRONG for it in comp)
                has_plain  = any(it.odor_level is not OdorLevel.STRONG for it in comp)
                odor_mix_risk = "⚠️" if (has_strong and has_plain) else " "
                lines.append(f"[Comp {idx}] {odor_mix_risk}")
                for it in comp:
                    lines.append(f"  - {it.name} [{it.category.name}]  {fmt_simple(it)}")

        if self.phase == "done":
            spent = self.order.spent_time
            late  = max(0.0, spent - self.order.time_limit_s)
            lines.append(f"Delivered. Spent={spent:.1f}s, Late={late:.1f}s.")

        self.viewer.set_info_text("\n".join(lines))

    # ---------- 取餐：装袋（自动分配隔层） ----------
    def _load_bag(self):
        self.bag = InsulatedBag(
            insulation_factor=0.70,
            num_compartments=4,
            has_divider=True
        )
        self.bag.start_new_order()
        for it in self.items:
            self.bag.add(it)  # 自动放到当前最空的隔层

    def _now(self):
        return self.sim_time


def main():
    app = QApplication(sys.argv)

    viewer = CityViewer(
        WORLD_JSON, BUILDING_DEFS,
        title="Order + Rider + Food (High-Level JSON)"
    )
    city_map = build_city_map(CONFIG)

    # 放大信息区
    try:
        viewer.set_info_area_height(220)
        viewer.set_info_font_point_size(11)
    except Exception:
        pass

    sim = SimRunner(viewer, city_map)

    viewer.add_button("New Order", sim.new_random_order)
    viewer.add_button("Start",     sim.start)
    viewer.add_button("Stop",      sim.stop)
    viewer.add_button("Time 0.5×", lambda: sim.set_time_scale(0.5))
    viewer.add_button("Time 1×",   lambda: sim.set_time_scale(1.0))
    viewer.add_button("Time 2×",   lambda: sim.set_time_scale(2.0))
    viewer.add_button("Time 4×",   lambda: sim.set_time_scale(4.0))
    viewer.add_button("Save Image",
        lambda: viewer.save_png(os.path.join(os.path.dirname(WORLD_JSON), "order_food_sim_view.png"))
    )

    sim.new_random_order()
    viewer.show()
    sys.exit(app.exec_())

if __name__ == "__main__":
    main()


c:\Users\Shadow\miniconda3\envs\citynav\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SystemExit: 0

c:\Users\Shadow\miniconda3\envs\citynav\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
